<a href="https://colab.research.google.com/github/Jijnash22/multimodal-parkinsons-ai/blob/main/notebooks/04_multimodal_fusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import drive
drive.mount('/content/drive')

BASE = "/content/drive/MyDrive/IIIT_Nagpur_PD_Project"

import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
# Load embeddings
voice_emb = np.load(BASE + "/features/voice_embeddings.npy")
face_emb = np.load(BASE + "/features/facial_embeddings.npy")

# Load models
voice_model = load_model(BASE + "/models/voice_model_32D.keras")
face_model  = load_model(BASE + "/models/face_pd_classifier.keras")

# Predict PD scores
PDv = voice_model.predict(voice_emb).flatten()
PDf = face_model.predict(face_emb).flatten()

print("Voice:", len(PDv))
print("Face :", len(PDf))


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
Voice: 81
Face : 40


In [12]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

PDv = scaler.fit_transform(PDv.reshape(-1,1)).flatten()
PDf = scaler.fit_transform(PDf.reshape(-1,1)).flatten()



In [13]:
PD_hand = pd.read_csv(BASE + "/features/hand_pd_scores.csv")["PD_score"].values
PD_hand


array([0.        , 0.52072478, 0.35071088, 0.3687345 , 1.        ,
       0.6197075 , 0.298603  , 0.9534077 ])

In [14]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

PDv = scaler.fit_transform(PDv.reshape(-1,1)).flatten()
PDf = scaler.fit_transform(PDf.reshape(-1,1)).flatten()



In [16]:
import glob, os
import numpy as np

hc_files = glob.glob(BASE + "/data/voice_dataset/HC/HC_AH/*.wav")
pd_files = glob.glob(BASE + "/data/voice_dataset/PD/PD_AH/*.wav")

y_voice = np.array([0]*len(hc_files) + [1]*len(pd_files))

print("Voice labels:", y_voice.shape, " PD:", y_voice.sum())



Voice labels: (81,)  PD: 40


In [17]:
y_face = np.array([0]*20 + [1]*20)
print("Face labels:", y_face.shape)



Face labels: (40,)


In [18]:
y_hand = np.array([0,0,0,0,1,0,0,1])
print("Hand labels:", y_hand)



Hand labels: [0 0 0 0 1 0 0 1]


In [12]:
!find "/content/drive/MyDrive" -type f | grep hand_pd


/content/drive/MyDrive/IIIT_Nagpur_PD_Project/features/hand_pd_scores.csv


In [14]:
PD_hand = pd.read_csv("/content/drive/MyDrive/IIIT_Nagpur_PD_Project/features/hand_pd_scores.csv")["PD_score"].values
PD_hand


array([0.        , 0.52072478, 0.35071088, 0.3687345 , 1.        ,
       0.6197075 , 0.298603  , 0.9534077 ])

In [16]:
import glob, os

hc_files = glob.glob(BASE + "/data/voice_dataset/HC/HC_AH/*.wav")
pd_files = glob.glob(BASE + "/data/voice_dataset/PD/PD_AH/*.wav")

y_voice = np.array([0]*len(hc_files) + [1]*len(pd_files))
len(hc_files), len(pd_files), len(y_voice), len(PDv)


(41, 40, 81, 81)

In [19]:
from sklearn.metrics import roc_auc_score

results = {}

results["Voice only"] = roc_auc_score(y_voice, PDv)
results["Face only"]  = roc_auc_score(y_face, PDf)
results["Hand only"]  = roc_auc_score(y_hand, PD_hand)

results["Voice + Face"] = roc_auc_score(y_face, (PDv[:40] + PDf) / 2)
results["Voice + Hand"] = roc_auc_score(y_hand, (PDv[:8] + PD_hand) / 2)
results["Face + Hand"]  = roc_auc_score(y_hand, (PDf[:8] + PD_hand) / 2)

results["All 3"] = roc_auc_score(
    y_hand,
    (PDv[:8] + PDf[:8] + PD_hand) / 3
)

results



{'Voice only': np.float64(0.901829268292683),
 'Face only': np.float64(0.5),
 'Hand only': np.float64(1.0),
 'Voice + Face': np.float64(0.6950000000000001),
 'Voice + Hand': np.float64(1.0),
 'Face + Hand': np.float64(1.0),
 'All 3': np.float64(1.0)}